In [ ]:
from __future__ import annotations
import polars as pl
import inspect
import numpy as np
from typing import Tuple

In [ ]:
from dsds.metrics import roc_auc
from dsds.transform import multicat_one_hot_encode

In [ ]:
pred = np.random.random(size=1_000_000) # .reshape(-1, 1) # Predictions
actual = np.random.random(size=1_000_000) # Actuals

In [ ]:
pl.show_versions()

In [ ]:
df = pl.DataFrame({
    "text1":["abc|ggg", "abc|sss", "ccc|abc"],
    "text2":["aaa|bbb", "ccc|aaa", "bbb|ccc"]
})
print(df)

In [ ]:
print(multicat_one_hot_encode(df, cols=["text1", "text2"], delimiter="|"))

In [ ]:
test = pl.DataFrame({
    "a":[1,1,1,1]
    , "b": [None, None , "a", "a"]
})
test

In [1]:
import dsds.sample as sa
import polars as pl
from sklearn.datasets import make_classification

orig_x, orig_y = make_classification(n_samples = 300_000, n_features = 50, n_informative = 25, n_redundant = 25)
df = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y))
target = "target"
features = df.columns
features.remove(target)

In [2]:
df.groupby("target").count()

target,count
i32,u32
0,149908
1,150092


In [26]:
test1 = sa.simple_downsample(df, pl.col("target") == 0, sample_frac=0.5)
test1.groupby("target").count()

target,count
i32,u32
0,74954
1,150092


In [27]:
test = df.with_columns(
    pl.when(pl.col("target") == 0).then(True).otherwise(False).alias("z")
).filter(
    (~pl.col("z"))
    | (pl.int_range(0, pl.count()).over("z").shuffle() < pl.count().over("z").max() * 0.5)
)

In [28]:
test.groupby("target").count()

target,count
i32,u32
0,75195
1,150092


In [37]:
%%timeit
_ = sa.simple_downsample(df.lazy(), pl.col("target") == 0, sample_frac=0.5).collect()

54.6 ms ± 744 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
%%timeit 
test = df.lazy().with_columns(
    pl.when(pl.col("target") == 0).then(True).otherwise(False).alias("z")
).filter(
    (~pl.col("z"))
    | (pl.int_range(0, pl.count()).over("z").shuffle() < pl.count().over("z").max() * 0.5)
).select(df.columns).collect()

28 ms ± 636 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
